In [23]:
import numpy as np
import itertools
import import_ipynb

from benchmark_functions import test_model, model_function  # Import function and class
from loss_functions import absolute_error  # Import function and class

In [24]:
# Defines the limits of the input itself, like in a function that depends on x we limit it from [0, 1].
# It only sets the grid of the space, IT HAS NO PROPERTIES*.
#class TargetSpace:
#class SampleSpace:
#class OriginalSpace:
class ConcreteSpace:
    def __init__(self, space_shape, limits, sample_method="same"):
        assert len(space_shape) == len(limits), "Each dimension must have corresponding limits"
        
        self.space_shape = space_shape
        self.limits = np.array(limits)
        self.sample_method = sample_method
        self.space = None  # Stores sampled points
        self.grid = None   # Stores full grid of combinations

    def generate_space(self):
        if self.sample_method == "same":
            # Create equally spaced points per dimension
            self.space = [np.linspace(low, high, num) for (low, high), num in zip(self.limits, self.space_shape)]
        elif self.sample_method == "monte_carlo":
            # Create random sampled points per dimension
            self.space = [np.random.uniform(low, high, num) for (low, high), num in zip(self.limits, self.space_shape)]
        else:
            raise ValueError("Invalid sample_method. Choose 'same' or 'monte_carlo'.")
        
    def generate_grid(self):
        if self.space is None:
            raise RuntimeError("Space must be generated first. Call generate_space()")

        # Generate all possible combinations of points in all dimensions
        self.grid = np.array(list(itertools.product(*self.space)))

    def get_space(self):
        return self.space
    
    def get_grid(self):
        if self.grid is None:
            raise RuntimeError("Grid has not been generated. Call generate_space() first.")
        return self.grid

    def change(self, new_shape=None, new_limits=None, new_method=None):
        if new_shape:
            assert len(new_shape) == len(self.limits), "New shape must match dimensions of limits"
            self.space_shape = new_shape
        if new_limits:
            assert len(new_limits) == len(self.space_shape), "New limits must match number of dimensions"
            self.limits = np.array(new_limits)
        if new_method:
            assert new_method in ["same", "monte_carlo"], "Invalid method"
            self.sample_method = new_method
        
        self.generate_space()  # Regenerate space and grid

In [25]:
# Define a 2D space (2 points on X-axis, 3 points on Y-axis)
space_shape = [2, 3,]
limits = [[-10, 10], [1, 2]]
sample_method = "same"  # Can also use "monte_carlo"

# Create the space
space = ConcreteSpace(space_shape, limits, sample_method)
space.generate_space()
space.generate_grid()

# Print sampled space
print("Generated Space:", space.get_space())

# Print full grid of combinations
print("Generated Grid:", space.get_grid())

Generated Space: [array([-10.,  10.]), array([1. , 1.5, 2. ])]
Generated Grid: [[-10.    1. ]
 [-10.    1.5]
 [-10.    2. ]
 [ 10.    1. ]
 [ 10.    1.5]
 [ 10.    2. ]]


In [26]:
# Defines the parameters available for the model to sample in a linear model we may have a and b varying from [0, 1]
# It only sets the grid of the space, IT HAS NO PROPERTIES*.
#class DualSpace:
#class SearchSpace:
class AbstractSpace:
    def __init__(self, space_shape, limits, sample_method="same"):
        assert len(space_shape) == len(limits), "Each dimension must have corresponding limits"
        
        self.space_shape = space_shape
        self.limits = np.array(limits)
        self.sample_method = sample_method
        self.space = None  # Stores sampled points
        self.grid = None   # Stores full grid of combinations

    def generate_space(self):
        if self.sample_method == "same":
            # Create equally spaced points per dimension
            self.space = [np.linspace(low, high, num) for (low, high), num in zip(self.limits, self.space_shape)]
        elif self.sample_method == "monte_carlo":
            # Create random sampled points per dimension
            self.space = [np.random.uniform(low, high, num) for (low, high), num in zip(self.limits, self.space_shape)]
        else:
            raise ValueError("Invalid sample_method. Choose 'same' or 'monte_carlo'.")
        
    def generate_grid(self):
        if self.space is None:
            raise RuntimeError("Space must be generated first. Call generate_space()")

        # Generate all possible combinations of points in all dimensions
        self.grid = np.array(list(itertools.product(*self.space)))

    def get_space(self):
        return self.space
    
    def get_grid(self):
        if self.grid is None:
            raise RuntimeError("Grid has not been generated. Call generate_space() first.")
        return self.grid

    def change(self, new_shape=None, new_limits=None, new_method=None):
        if new_shape:
            assert len(new_shape) == len(self.limits), "New shape must match dimensions of limits"
            self.space_shape = new_shape
        if new_limits:
            assert len(new_limits) == len(self.space_shape), "New limits must match number of dimensions"
            self.limits = np.array(new_limits)
        if new_method:
            assert new_method in ["same", "monte_carlo"], "Invalid method"
            self.sample_method = new_method
        
        self.generate_space()  # Regenerate space and grid


In [27]:
# Define a 2D space (2 points on X-axis, 3 points on Y-axis)
space_shape = [2, 3,]
limits = [[-10, 10], [1, 2]]
sample_method = "same"  # Can also use "monte_carlo"

# Create the space
space = AbstractSpace(space_shape, limits, sample_method)
space.generate_space()
space.generate_grid()

# Print sampled space
print("Generated Space:", space.get_space())

# Print full grid of combinations
print("Generated Grid:", space.get_grid())

Generated Space: [array([-10.,  10.]), array([1. , 1.5, 2. ])]
Generated Grid: [[-10.    1. ]
 [-10.    1.5]
 [-10.    2. ]
 [ 10.    1. ]
 [ 10.    1.5]
 [ 10.    2. ]]


In [28]:
# IMPORTANT: The search space is a N dimensional space in which the parameters represent some operation in a model, the operations are the PROPERTIES attributed to
# that search space, one search space can have many PROPERTIES, but one PROPERTY can have only be defined in the specific dimensions of that search space
# If the AI model has the exact complexity of the object we want to understant, the search space and the sample space have the same dimension
# If the AI model has the same property of the object we want to understant, the properties of that space are the same, in this case we have a perfect model
# A perfect model means, given some input in the search and sample space, the output will be the same
# With that in mind, perhaps oversized models can still not overfit if the specific output is orthogonal to the effects given by the extra parameters
class ModelProcessor:
    def __init__(self, model_function):
        self.model_function = model_function

    def process_single(self, concrete_item, abstract_item):
        return self.model_function(concrete_item, abstract_item)

    def process_concrete_full_abstract_one(self, concrete_grid, abstract_item):
        return [self.model_function(concrete, abstract_item) for concrete in concrete_grid]

    def process_abstract_full_concrete_one(self, concrete_item, abstract_grid):
        return [self.model_function(concrete_item, abstract) for abstract in abstract_grid]

    def compare_models(self, concrete_grid, abstract_model_1, abstract_model_2, loss_function, return_mean=False):
        results_1 = self.process_concrete_full_abstract_one(concrete_grid, abstract_model_1)
        results_2 = self.process_concrete_full_abstract_one(concrete_grid, abstract_model_2)

        loss_values = [loss_function(r1, r2) for r1, r2 in zip(results_1, results_2)]
        return np.mean(loss_values) if return_mean else loss_values
    
    def evaluate_against_target(self, concrete_grid, target_model, abstract_grid, loss_function):
        loss_results = []
        
        for abstract_candidate in abstract_grid:
            loss_value = self.compare_models(concrete_grid, target_model, abstract_candidate, loss_function, return_mean=True)
            loss_results.append(loss_value)

        return loss_results
    
    def compare_concrete_models(self, concrete_model_1, concrete_model_2, abstract_grid, loss_function, return_mean=False):
        results_1 = self.process_abstract_full_concrete_one(concrete_model_1, abstract_grid)
        results_2 = self.process_abstract_full_concrete_one(concrete_model_2, abstract_grid)

        loss_values = [loss_function(r1, r2) for r1, r2 in zip(results_1, results_2)]
        return np.mean(loss_values) if return_mean else loss_values
    
    def compare_concrete_models_iteration(self, concrete_model_1, target_model, abstract_grid, loss_function, return_mean=False):
        results_1 = self.process_abstract_full_concrete_one(concrete_model_1, abstract_grid)    # Do not pass the grid, pass only the correct model
        target = self.process_single(concrete_model_1, target_model)

        loss_values = [loss_function(target, r1) for r1 in results_1]
        return np.mean(loss_values) if return_mean else loss_values
    
    def evaluate_concrete_against_target(self, concrete_grid, target_model, abstract_grid, loss_function):
        loss_results = []
        
        for concrete_candidate in concrete_grid:
            loss_value = self.compare_concrete_models_iteration(concrete_candidate, target_model, abstract_grid, loss_function, return_mean=True)
            loss_results.append(loss_value)

        return loss_results

In [29]:
# Create the spaces
concrete_space = ConcreteSpace(space_shape, limits, sample_method)
concrete_space.generate_space()
concrete_space.generate_grid()

abstract_space = AbstractSpace(space_shape, limits, sample_method)
abstract_space.generate_space()
abstract_space.generate_grid()

# Create the model processor
model_processor = ModelProcessor(test_model)

# Process single concrete and abstract item
result_single = model_processor.process_single(concrete_space.get_grid()[0], abstract_space.get_grid()[0])

# Process full concrete grid with one abstract item
result_concrete_full_abstract_one = model_processor.process_concrete_full_abstract_one(concrete_space.get_grid(), abstract_space.get_grid()[0])

# Process one concrete item with full abstract grid
result_abstract_full_concrete_one = model_processor.process_abstract_full_concrete_one(concrete_space.get_grid()[0], abstract_space.get_grid())

# Print results
print("Single Concrete & Single Abstract:", result_single)
print("Full Concrete Grid & Single Abstract:", result_concrete_full_abstract_one)
print("Single Concrete & Full Abstract Grid:", result_abstract_full_concrete_one)


Single Concrete & Single Abstract: [-18.0, 0.0]
Full Concrete Grid & Single Abstract: [[-18.0, 0.0], [-17.5, -5.0], [-17.0, -10.0], [2.0, 20.0], [2.5, 25.0], [3.0, 30.0]]
Single Concrete & Full Abstract Grid: [[-18.0, 0.0], [-17.5, 5.0], [-17.0, 10.0], [2.0, -20.0], [2.5, -25.0], [3.0, -30.0]]


In [30]:
# Create the model processor
model_processor = ModelProcessor(model_function)

# Create spaces
concrete_space = ConcreteSpace(space_shape, limits, sample_method)
concrete_space.generate_space()
concrete_space.generate_grid()

abstract_space = AbstractSpace(space_shape, limits, sample_method)
abstract_space.generate_space()
abstract_space.generate_grid()

# Extract two different abstract models
abstract_model_1 = abstract_space.get_grid()[0]  # First model parameter
abstract_model_2 = abstract_space.get_grid()[1]  # Second model parameter

# Get model outputs
results_1 = model_processor.process_concrete_full_abstract_one(concrete_space.get_grid(), abstract_model_1)
results_2 = model_processor.process_concrete_full_abstract_one(concrete_space.get_grid(), abstract_model_2)

# Compare both models (return full loss list)
loss_list = model_processor.compare_models(concrete_space.get_grid(), abstract_model_1, abstract_model_2, absolute_error, return_mean=False)

# Compare both models (return mean loss)
mean_loss = model_processor.compare_models(concrete_space.get_grid(), abstract_model_1, abstract_model_2, absolute_error, return_mean=True)

# Print results
print("\nModel 1 Outputs:", results_1)
print("Model 2 Outputs:", results_2)
print("Loss list between models:", loss_list)
print("Loss mean between models:", mean_loss)



Model 1 Outputs: [-18.0, -17.5, -17.0, 2.0, 2.5, 3.0]
Model 2 Outputs: [-17.5, -17.0, -16.5, 2.5, 3.0, 3.5]
Loss list between models: [0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
Loss mean between models: 0.5


In [31]:
# Extract two different abstract models
concrete_model_1 = concrete_space.get_grid()[0]  # First model parameter
concrete_model_2 = concrete_space.get_grid()[1]  # Second model parameter

# Get model outputs
results_1 = model_processor.process_abstract_full_concrete_one(concrete_model_1, abstract_space.get_grid())
results_2 = model_processor.process_abstract_full_concrete_one(concrete_model_2, abstract_space.get_grid())

# Compare both models (return full loss list)
loss_list = model_processor.compare_concrete_models(concrete_model_1, concrete_model_2, abstract_space.get_grid(), absolute_error, return_mean=False)

# Compare both models (return mean loss)
mean_loss = model_processor.compare_concrete_models(concrete_model_1, concrete_model_2, abstract_space.get_grid(), absolute_error, return_mean=True)

# Print results
print("\nModel 1 Outputs:", results_1)
print("Model 2 Outputs:", results_2)
print("Loss list between models:", loss_list)
print("Loss mean between models:", mean_loss)


Model 1 Outputs: [-18.0, -17.5, -17.0, 2.0, 2.5, 3.0]
Model 2 Outputs: [-17.5, -17.0, -16.5, 2.5, 3.0, 3.5]
Loss list between models: [0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
Loss mean between models: 0.5


In [32]:
# Select a fixed target model (abstract representation)
target_model = abstract_space.get_grid()[0]  # This is the reference model

# Evaluate each abstract model in the grid against the target model
loss_values = model_processor.evaluate_against_target(
    concrete_grid=concrete_space.get_grid(),
    target_model=target_model,
    abstract_grid=abstract_space.get_grid(),
    loss_function=absolute_error
)

# Print results
print("\nLoss values for each abstract model compared to the target model:")
print(loss_values)


Loss values for each abstract model compared to the target model:
[0.0, 0.5, 1.0, 20.0, 20.5, 21.0]


In [33]:
# Select a fixed target model (abstract representation)
target_model = abstract_space.get_grid()[0]  # This is the reference model

# Evaluate each abstract model in the grid against the target model
loss_values = model_processor.evaluate_concrete_against_target(
    concrete_grid=concrete_space.get_grid(),
    target_model=target_model,
    abstract_grid=abstract_space.get_grid(),
    loss_function=absolute_error
)

# Print results
print(loss_values)

[10.5, 10.5, 10.5, 10.5, 10.5, 10.5]
